# Embedding
![rag_embedding](figures/rag_embedding.png)

- 분할된 텍스트를 벡터 표현(임베딩 벡터)으로 변환한다.
- LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원하며, 동일한 인터페이스로 사용할 수 있다.
- [임베딩모델의 메서드](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.embeddings.Embeddings.html#langchain_core.embeddings.embeddings.Embeddings)

    - **`embed_documents(texts: List[str])`**
        - 여러 문서를 받아 벡터화(임베딩)한다.
        - Context를 벡터화 할 때 사용한다.
    - **`embed_query(text: str)`**
        - 하나의 문자열(문서)을 받아 벡터화한다.
        - Query를 벡터화 할 때 사용한다.


In [6]:
docs = [
        "나는 고양이와 개 중 반려동물로 개를 키우고 싶습니다.",
        "이 강아지 품종은 진도개 입니다. 국제 표준으로 중대형견으로 분류되며 다리가 길어 체고가 높은 편에 속합니다.",
        "日本の市内バスの運賃は主に距離によって決まり、地域やバス会社によって異なる場合があります",                  # 일본의 시내버스 요금은 주로 거리에 따라 결정되며, 지역 및 버스 회사에 따라 다를 수 있습니다.
        "Bus fares in the United States vary from city to city, but are generally around $2.90 for a regular bus.",  # 미국의 버스 요금은 도시마다 다르지만, 일반적으로 정기 버스의 경우 2.90달러 정도입니다.
        "광역버스 요금은 일반 3000원, 청소년은 1800원, 어린이 1500원 입니다.", 
]

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
#####################################
# OpenAI의 Embedding 모델.
#####################################

import numpy as np
from langchain_openai import OpenAIEmbeddings

# e_model_id = "text-embedding-3-small" # "text-embedding-3-large"
e_model_id = "text-embedding-3-large"
embedding_model = OpenAIEmbeddings(model=e_model_id)

In [50]:
#####################################
# Ollama Embedding 모델
#####################################

from langchain_ollama import OllamaEmbeddings

e_model_id = "bge-m3"
embedding_model = OllamaEmbeddings(model=e_model_id)

In [63]:
#####################################
# Hungging_face Embedding 모델
#####################################
# Hugging_face : Model -> task - NLP -> Feature Extraction

from langchain_huggingface import HuggingFaceEmbeddings

e_model_id = "intfloat/multilingual-e5-large"
embedding_model = HuggingFaceEmbeddings(model=e_model_id)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\tjddm\miniconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjddm\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [64]:
# 문서들을 embedding
embedded_docs = embedding_model.embed_documents(docs)

In [65]:
# import numpy as np

np.shape(embedded_docs), type(embedded_docs)
# # (5 : 문서 개수, 1536 : 개별 문서의 Embedding 차원)

((5, 1024), list)

In [66]:
# query = "당신이 좋아하는 동물은 무엇인가요?"
query = "성인 버스요금은 얼마인가요?"
embedded_query = embedding_model.embed_query(query)
np.shape(embedded_query), type(embedded_query)

((1024,), list)

In [54]:
import numpy as np

def cosine_similarity(v1 : np.ndarray | list, v2 : np.ndarray | list) -> float:
    # v1과 v2의 코사인 유사도 계산.
    v1 = np.array(v1)
    v2 = np.array(v2)
    return (v1 @ v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [67]:
# embedded_query와 embedded_docs 간의 유사도 계산.

for i, ev in enumerate(embedded_docs):
    print(f"{i+1}. {cosine_similarity(ev, embedded_query)}")

1. 0.7523771420277428
2. 0.7313870736100034
3. 0.767811236946179
4. 0.7869861337661619
5. 0.8564513211760618


# 벡터 데이터베이스(Vector Database)
- Embedding 된 문서를 Vector Database(Vector Store)에 저장한다.
- 이후 질문(Query)와 관련된 내용을 유사도를 이용해 검색해 질문과 함께 prompt로 만든다. (Retrieve)

![rag_vector_store](figures/rag_vector_store.png)

## 벡터 데이터베이스란
- 벡터 임베딩을 저장하고 관리하는 데이터베이스를 의미한다.
- 모든 데이터는 적절한 임베딩 모델을 활용하면 임베딩 벡터로 변환할 수 있다. 이렇게 변환된 임베딩 벡터를 벡터 데이터베이스에 저장하면 **임베딩 벡터 간의 거리 계산을 통해 데이터 간 유사도를 검색할 수 있다.**
    - **이미지, 텍스트, 음성 등 비정형 데이터**를 임베딩 모델로 **벡터화한 뒤 데이터베이스에 저장**한다.
    - 벡터 간의 **유사도 계산**을 통해 연관성 있는 데이터나 유사한 데이터를 효과적으로 검색할 수 있다.
    - 좋은 검색 결과를 위해서는 벡터의 품질이 중요하다. 그래서 **임베딩 모델(Embedding Model)을 잘 선택하는 것이 중요**하다.
- 벡터 데이터베이스는 이러한 벡터 간 거리 계산에 특화된 데이터베이스다.

## 주요 특징

- **고차원 벡터 저장**
  -  벡터 데이터베이스는 수백에서 수천 차원에 이르는 벡터 데이터를 효율적으로 저장하고 관리한다. 
  -  전통적인 데이터베이스로는 어려운 고차원 벡터 간 유사도 검색을 효율적으로 수행한다.
- **유사성 기반 검색**
  -  벡터 간의 거리를 측정하여 유사한 데이터를 빠르게 검색할 수 있다. 
  -  일반적으로 사용되는 거리계산기법은 다음과 같다.
     - 코사인 유사도(Cosine Similarity)
     - 유클리드 거리(Euclidean Distance)
     - 맨하탄 거리(Manhattan Distance) 
- 비정형 데이터 처리: 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 변환하여 저장하고, 이러한 데이터를 효과적으로 검색할 수 있다.

## 벡터 데이터베이스와 딥러닝
- 벡터 데이터베이스는 딥러닝 기술의 발전과 깊은 관련이 있다.
- 딥러닝 모델은 학습 과정에서 데이터의 특징을 추출하는 방법을 함께 학습한다. 충분한 데이터를 학습한 딥러닝 모델은 **데이터의 특성을 설명하는 특성 벡터(feature vector)를 효과적으로 생성**할 수 있다.
- 이때 추출된 특성 벡터는 고차원 데이터(RAW Data)를 저차원 공간에서 표현한 **임베딩 벡터**다.
    - > **임베딩**은 고차원 데이터를 저차원 공간으로 변환하여 표현하는 방법으로, 정보 손실을 최소화하면서 데이터 간의 의미 있는 관계를 벡터 공간에서 유지한다.
- 딥러닝 모델로 추출한 데이터의 특징(feature vector)을 임베딩 공간에 배치하면, 비슷한 데이터는 가까이, 그렇지 않은 데이터는 멀리 배치된다.
- 이러한 특성을 활용하면 임베딩 벡터 간의 거리를 계산해 유사한 데이터를 효과적으로 검색할 수 있다. 벡터 데이터베이스는 이러한 임베딩 벡터의 특성을 기반으로 개발되었다.
- 딥러닝 기술의 발전과 폭넓은 활용으로 임베딩 데이터의 사용이 증가하면서, 이를 저장하고 관리하는 기능에 특화된 데이터베이스에 대한 수요도 증가해 다양한 벡터 데이터베이스가 등장했다.

## 벡터 데이터베이스의 주요 기능
1. **저장**  
   - 이미지, 텍스트, 음성 등 **비정형 데이터**를 임베딩 모델을 통해 벡터로 변환한 뒤 벡터 데이터베이스에 저장한다.
2. **검색**  
   - 검색하려는 데이터를 임베딩 모델로 변환한 뒤, 벡터 데이터베이스에서 **유사도를 기반**으로 검색한다.
3. **결과 반환**  
   - 벡터 데이터베이스는 저장된 벡터 중 검색 쿼리 임베딩과 가장 가까운 벡터를 찾아 반환한다.

## LLM과 벡터 데이터베이스
- ChatGPT(LLM)의 등장 이후 벡터 데이터베이스는 폭발적인 주목을 받았다.
- 임베딩 벡터의 유사도를 기반으로 문서를 검색하는 RAG(Relevant Augmented Generation) 기술은 LLM의 환각(할루시네이션) 현상을 줄이고, LLM을 추가 학습하지 않고도 최신 정보를 효율적으로 활용할 수 있는 핵심 기법으로 자리 잡았다.
   


## 벡터 데이터베이스 종류
![img](figures/vector_database.png)

<<https://blog.det.life/why-you-shouldnt-invest-in-vector-databases-c0cd3f59d23c>>

### 주요 벡터 데이터베이스 종류
- **Pinecone**
    - 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 간단한 API를 통해 벡터 데이터를 관리할 수 있다.  
    - 자동 확장성과 고가용성을 제공하며, 실시간 데이터 수집과 유사성 검색에 최적화되어 있다.
    - 가장 쉽게 시작할 수 있는 관리형 서비스를 제공한다.
- **Chroma**
    - 벡터 임베딩을 효율적으로 저장하고 검색할 수 있는 오픈소스 데이터베이스로, AI 및 머신러닝 애플리케이션에 최적화되어 있다.
    - 대규모 임베딩 저장에 최적화되어 있다.
- **FAISS**
    - Facebook AI에서 개발한 고성능 벡터 검색 라이브러리로, 고차원 벡터의 효율적인 유사성 검색을 위해 최적화되어 있다.
    - GPU를 활용해 계산 성능을 높이며, 벡터 양자화 기술을 활용하여 메모리 사용을 최적화한다.
    - 근사 최근접 이웃 검색(ANNS)에 최적화되어 있다.
- **Milvus**
    - 오픈소스 벡터 데이터베이스로, 대규모 벡터 데이터를 효율적으로 저장하고 검색할 수 있다.  
    - 분산 아키텍처를 채택하여 확장성이 뛰어나며, IVF_PQ, DiskANN 등 다양한 인덱싱 알고리즘을 지원한다.
    - 대규모 데이터셋 처리에 가장 적합한 솔루션이다.
- **Weaviate**
    - 오픈소스 벡터 데이터베이스로, 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 저장하고 검색할 수 있다.  
    - GraphQL API를 통해 접근 가능하며, 내장된 머신러닝 모듈을 통해 가장 강력한 의미론적 검색 기능을 제공한다.
- **Qdrant**
    - Rust로 개발된 고성능 벡터 검색 엔진으로, 실시간 근사 최근접 이웃 검색을 제공한다.  
    - 추천 시스템에 특화되어 있으며, 벡터 임베딩 저장과 유사도 쿼리를 효율적으로 수행한다.
- **Elasticsearch**
    - HNSW 알고리즘을 사용하여 벡터 검색을 구현하는 검색 엔진이다.
    - 전통적인 검색 기능과 벡터 검색을 효과적으로 결합할 수 있어, 하이브리드 검색에 가장 적합하다.
- **PGVector**
    - PostgreSQL의 확장 모듈로, 벡터 데이터를 저장하고 유사성 검색을 수행할 수 있게 해준다.  
    - SQL과 통합된 벡터 연산이 가능하며, L2 거리, 코사인 거리, 내적 등 다양한 거리 측정 방식을 지원한다.


# Langchain - Vector Store 연동 
- Langchain은 다양한 벡터 데이터베이스와 연동할 수 있다.
- 벡터 데이터베이스 마다 API가 다르기 때문에, Langchain을 사용하면 동일한 interface로 사용할 수 있다.

## **VectorStore**
- Langchain이 지원하는 모든 벡터 데이터베이스는 **VectorStore** 인터페이스를 구현한다.
- 그래서 Langchain에서는 벡터 데이터베이스를 **Vector Store** 라고 한다.
- https://python.langchain.com/docs/integrations/vectorstores/

### Vector Store 연결
- Vector DB와 연결하는 메소드
- `VectorStore.from_document()`
  - Document들을 insert 하면서 연결.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - documents: insert할 문서들을 list[Document]로 전달.
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
-`VectorStore()`
  - vector db와 연결만 한다.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
## InMemoryVectorStore
- langchain에서 제공하는 메모리 기반 벡터 데이터베이스이다.
- Data들을 Dictionary를 사용해 메모리에 저장하며, 검색 할 때 코사인 유사도(cosine similarity)를 계산하여 조회한다.

In [68]:
from dotenv import load_dotenv

load_dotenv()

True

In [73]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# VectorStore 생성시 Embedding 모델을 넣어 생성한다.

vector_store = InMemoryVectorStore(
    embedding = embedding_model
)

In [75]:
# 문서 정의

from langchain_core.documents import Document

d1 = Document(
    id = "1", # 문서 ID(식별자)
    page_content="Apple, Pear, Watermelon", # 문서 내용
    metadata={"category":"fruit"}, # 문서 정보
)

d2 = Document(
    id = "2",
    page_content="Python, C++, Java, C#, Rust",
    metadata={"category":"IT"}
)

d3 = Document(
    id = "3",
    page_content="Football, Baseball, Basketball",
    metadata={"category":"Sports"}
)

docs = [d1,d2,d3]

# Vector DB에 저장.

vector_store.add_documents(documents=docs)

['1', '2', '3']

In [76]:
# DB와 연결하면서 document들을 insert/upsert

vector_store2 = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding = embedding_model
)

In [79]:
query = "SQL"

result = vector_store.similarity_search_with_score( # 검색한 결과 + 유사도 점수
# result = vector_store.similarity_search(
    query=query, # 찾을 질문.
    k=2, # 몇개 문서를 찾을지 지정
)

In [80]:
result

[(Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#, Rust'),
  0.29180902701976313),
 (Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.2635361695895074)]

# 실습
- data/olympic.txt
1. text loading
2. text split
3. embedding + vector store(InMemoryVectorStore)에 저장
4. query(질의)

In [81]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

path = "data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
docs = loader.load() 

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding = embedding_model
)
query = "올림픽"

result = vector_store.similarity_search(
    query=query,
    k=2
)

In [82]:
result

[Document(id='39f65c74-80db-4a32-aad8-da850bc96456', metadata={'source': 'data/olympic.txt'}, page_content='올림픽\n올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.\n또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기

## MMR(최대 한계 관련성-Maximal Marginal Relevance) 알고리즘 적용
최대 한계 관련성(Maximal Marginal Relevance, MMR) 알고리즘은 정보 검색 및 요약에서 검색 결과의 **관련성**과 **다양성**을 동시에 고려하여 최적의 결과를 제공하는 방법이다. 
이 알고리즘은 사용자 쿼리와의 관련성을 최대화하면서도 중복 정보를 최소화하여 다양한 정보를 제공하는 것을 목표로 한다.

1. **관련성과 다양성의 균형 조절**: MMR은 사용자 쿼리와 문서 간의 유사성 점수와 이미 선택된 문서들과의 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한다. 이를 통해 관련성이 높으면서도 중복되지 않는 문서를 선택한다.

2. **수학적 정의**
   $$
   \text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')
   $$

   - $\text{Sim}(d, Q)$: 문서 $d$와 쿼리 $\text{Q}$ 사이의 유사성. (문서 유사성 계산)
   - $\max_{d' \in D'} \text{Sim}(d, d')$: 문서 $d$와 이미 선택된 문서 집합 $D'$ 중 가장 유사한 문서와의 유사성. (문서 다양성 계산)
   - $\lambda$: 유사성과 다양성의 중요도를 조절하는 매개변수(parameter)
3. **적용 분야**: MMR은 정보 검색, 추천 시스템, 문서 요약 등에서 활용된다. 특히 LLM 검색에서 성능 향상이 입증되었다.

### `vectorStore.max_marginal_relevance_search()` 메소드
  - MMR 알고리즘을 적용한 검색을 수행한다.
  - **파라미터**
    - **query**: 사용자로부터 입력받은 검색 쿼리
    - **k**: 최종적으로 선택할 문서의 수
    - **fetch\_k**: MMR 알고리즘 적용 시 고려할 상위 문서의 수
    - **lambda_mult**: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절하는 매개변수. $\lambda = 1$이면 유사성만 고려하고, $\lambda = 0$이면 다양성만을 최대화한다.
    - **filter**: 검색 결과를 필터링할 조건을 지정한다.
